# Missile vs Bird Classification using SVM + Fog Removal + HOG

In [1]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# STEP 2: Install Required Libraries
!pip install -q opencv-python scikit-image scikit-learn matplotlib

In [ ]:
# STEP 3: Import Libraries
import os
import cv2
import joblib
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt

In [ ]:
# STEP 4: Fog Removal Function
def remove_fog_clahe(gray_img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(gray_img)

In [ ]:
# STEP 5: Load Trained SVM Model
svm = joblib.load("/content/svm_model.pkl")  # ✅ Your model path

In [ ]:
# STEP 6: Predict + Show Image
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to load: {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fog_removed = remove_fog_clahe(gray)
    resized = cv2.resize(fog_removed, (128, 128))
    features = hog(resized, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys')

    prediction = svm.predict([features])[0]
    label = "Missile" if prediction == 1 else "Bird"

    plt.imshow(fog_removed, cmap='gray')
    plt.title(f"Prediction: {label}")
    plt.axis('off')
    plt.show()

    print(f"✅ {os.path.basename(image_path)} ➤ Prediction: {label}")
    return label

In [ ]:
# STEP 7: Predict All Images in a Folder
def predict_all_in_folder(test_folder_path):
    print(f"\n📁 Predicting images in folder: {test_folder_path}")
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

    for file in os.listdir(test_folder_path):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            full_path = os.path.join(test_folder_path, file)
            predict_image(full_path)

In [ ]:
# ✅ STEP 8: SET YOUR TEST IMAGE FOLDER PATH HERE:
test_folder = "/content/drive/MyDrive/drdo_project/testimage"
predict_all_in_folder(test_folder)